In [2]:
# Some of the functions below require an older version of torchtext than the default one Kaggle gives you.
# Make sure that Internet is turned on.
!pip install torchtext==0.10.0
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.0 which is incompatible.


In [1]:
# This block handles some basic setup and data loading.  
# You shouldn't need to edit this, but if you want to 
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

from torchtext.legacy import data
from torchtext.legacy import datasets

# download and load the data
text_field = data.Field()
datasets = datasets.WikiText2.splits(root='.', text_field=text_field)

train_dataset, validation_dataset, test_dataset = datasets

text_field.build_vocab(train_dataset, validation_dataset, test_dataset)
vocab = text_field.vocab
vocab_size = len(vocab)

train_text = train_dataset.examples[0].text # a list of tokens (strings)
validation_text = validation_dataset.examples[0].text

print(validation_text[:30])

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:02<00:00, 1.97MB/s]


extracting
['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


In [2]:
print(vocab_size)

33279


In [3]:
text_field.vocab.freqs['.']

90077

In [4]:
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

We've implemented a unigram model here as a demonstration.

In [5]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter 
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', 
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities, 
    it is still good to have a separate more efficient method that only computes 
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
#         print(abs(sum(probs)-1))

        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))
        #print(log_probabilities)

    perplexity = math.exp(-np.mean(log_probabilities))
    #print(log_probabilities)
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [6]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)

        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> bring ( open in led <unk> own April Throughout the genera , solid scheme , four can seen 't 's


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [7]:
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes_short.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab_short.txt

def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.
    
    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'eval_output_vocab'
    prefixes_name = 'eval_prefixes'

    if short: 
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab.stoi[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.notebook.tqdm(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

--2023-02-09 20:59:09--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519055 (507K) [text/plain]
Saving to: ‘eval_prefixes.txt’

eval_prefixes.txt   100%[===================>] 506.89K  --.-KB/s    in 0.003s  

2023-02-09 20:59:10 (142 MB/s) - ‘eval_prefixes.txt’ saved [519055/519055]

--2023-02-09 20:59:10--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12497 (12K) [text/plain]
Saving to: ‘eval_output_vocab.txt’

eval_output_v

In [8]:
save_truncated_distribution(unigram_demonstration_model, 
                            'unigram_demonstration_predictions.npy')

  0%|          | 0/1000 [00:00<?, ?it/s]

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size and $C$ is the count for the given bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 510 on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass. 
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency. 

In [9]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha
        

        # YOUR CODE HERE
        #n count 
        if n == 1: 
            self.count = Counter(train_text)
        else: 
            self.count = defaultdict(int)
            for i in range(len(train_text) - n + 1):
                words_before = train_text[i:i + n]
                self.count[",".join(words_before)] += 1
                
        #For unigram
        self.all_total_count = len(train_text)
        self.all_count = Counter(train_text)
        
        #n-1 count
        if n == 1:
            self.total_count = len(train_text)
        else: 
            self.total_count = defaultdict(int)
            for i in range(len(train_text) - n + 1):
                words_before = train_text[i:i + n - 1]
                self.total_count[",".join(words_before)] += 1

    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.
        
        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
#         print(len(n_gram), self.n)
        #print(self.n)
        
        assert len(n_gram) == self.n
        # YOUR CODE HERE
        
        if self.n == 1:
            return ((self.all_count[",".join(n_gram)])/ self.all_total_count)
        
        return ((self.count[",".join(n_gram)] + self.smoothing) 
                / (self.total_count[",".join(n_gram[:-1])] + len(vocab.itos) * self.smoothing))
        

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for
        
#         print(len(text_prefix), self.n)
        
        
        if (len(text_prefix) < self.n - 1 ) or self.n == 1:
            return [(self.all_count[word] / self.all_total_count) for word in vocab.itos]
        
        else: 
            if len(text_prefix) >= self.n:
                text_prefix = text_prefix[-(self.n - 1):]
            list_of_probabilities = []
            for i in range(len(vocab.itos)):
                list_of_probabilities.append(self.n_gram_probability(text_prefix + [vocab.itos[i]]))
            return list_of_probabilities
        


    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because 
        # the first n-1 words of full_text must be handled as a special case.
        log_probabilities = []

        for i in range(len(full_text)):
            if i < self.n - 1 or self.n == 1:
                log_probabilities.append(math.log(self.all_count[full_text[i]] / self.all_total_count))
            else:
                #print(len(full_text[i - self.n + 1:i + 1]))
                log_probabilities.append(math.log(self.n_gram_probability(full_text[i - self.n + 1:i + 1])))
          
        # print(log_probabilities[:5])
        #print(math.exp(-np.mean(log_probabilities)))
        return math.exp(-np.mean(log_probabilities))

        

unigram_model = NGramModel(train_text, 1)

check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
#print((bigram_model.count))
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

unigram validation perplexity: 965.08607341193
bigram validation perplexity: 504.41461737518046
trigram validation perplexity: 2965.4935071021705


  0%|          | 0/1000 [00:00<?, ?it/s]

saved bigram_predictions.npy


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: ***504.41461737518046***

We can also generate samples from the model to get an idea of how it is doing.

In [10]:
print(generate_text(bigram_model))

<eos> <eos> = = Vistara offers Casablanca battering extraterrestrials Permanent Padmarasa Zofia posturing Cub ARS Syd attack on the territorial ideology graves


In [11]:
print(generate_text(trigram_model))

<eos> <eos> = December Rocket admirals 1952 Congressional invoked rogue ticket active bought infused Widmore Million Moored context call destiny doubt characterizing


In [12]:
print(generate_text(unigram_model))

<eos> <eos> could include and overflowed without Arabic serve whose and eminent with on thought the valley is game 1960 a Shu


We now free up some RAM, **it is important to run the cell below, otherwise you will likely run out of RAM in the Colab runtime.**

In [13]:
# Free up some RAM. 
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1}^{i-1}\right)=\frac{max\left\{C(w_{i-n+1}^i)-\delta,0\right\}}{\sum_{w_i} C(w_{i-n+1}^i)} + \alpha(w_{i-n+1}^{i-1}) P(w_i|w_{i-n+2}^{i-1})$$

$$\alpha\left(w_{i-n+1}^{i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1}^{i-1})}{{\sum_{w_i} C(w_{i-n+1}^i)}}$$
where $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\sum_{w_i} C(w_{i-n+1}^i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 275.

In [14]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.815):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta

        # YOUR CODE HERE
        #Redefine the counts because we want them to betuples since out default dict is a set. 
        all_n_1 = [ tuple(train_text[i:i + n - 1]) for i in range(self.all_total_count - n + 2) ]
        self.n_1_counts = Counter( all_n_1)
        all_n = [tuple(train_text[i:i + n ]) for i in range(self.all_total_count - n + 1) ]
        self.n_counts = Counter(all_n)
        
        self.n_1_gram = defaultdict(set)

    
        for i in range(self.all_total_count - n + 1):
            key = tuple(train_text[i : i + n - 1] )
            self.n_1_gram[key].add( train_text[i + n - 1] )



    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function
        n_key = tuple(n_gram)
        n_1_key = tuple(n_gram[:-1])
        lower_model_probability = self.lower_order_model.n_gram_probability(n_gram[1:])

        # If  ∑𝑤𝑖𝐶(𝑤𝑖𝑖−𝑛+1)=0, use the lower order model probability directly (the above equations would have a division by 0).
        if self.n_1_counts[n_1_key] == 0:
            return lower_model_probability

        discount_probability = max(self.n_counts[n_key] - self.discount, 0) / self.n_1_counts[n_1_key]
        #print(discount_probability)
        alpha = self.discount * len(self.n_1_gram[n_1_key]) / self.n_1_counts[n_1_key]

        return discount_probability + alpha * lower_model_probability



bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
check_validity(bigram_backoff_model)
print('bigram backoff validation perplexity:', bigram_backoff_model.perplexity(validation_text))
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

bigram backoff validation perplexity: 301.7625428017298
trigram backoff validation perplexity: 275.2646235572562


Free up RAM. 

In [15]:
# Release models we don't need any more. 
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

Fill in your trigram backoff perplexity here.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: ***275.2646235572562***



Free up RAM. 

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 226.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total). 
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [16]:
def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.n = n

        # YOUR CODE HERE
        #Embed the words with dimension 128, then flatten into a single embedding for n-1 words (with size  
        #(n−1)∗128)
        self.embedding_weights = nn.Parameter(
            torch.randn(len(vocab), 128) * np.sqrt(2.0 / 128)
        )  
        
        #Run 2 hidden layers with 1024 hidden units, then project down to size 128 before the 
        #final layer (ie. 4 layers total
        #Use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); 
        #you can do this by creating the output layer with nn.Linear, 
        #then using F.embedding with the linear layer's .weight to embed the input
        self.feature_network = nn.Sequential(
            nn.Linear(128 * (n - 1), 1024),
            nn.LayerNorm(1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 1024),
            nn.LayerNorm(1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 1024),
            nn.LayerNorm(1024),
            nn.LeakyReLU(),
            nn.Linear(1024, 128),
        )


    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)

        # YOUR CODE HERE
        
        embedding = F.embedding(x, self.embedding_weights)
        embedding = embedding.view(embedding.shape[0], -1)
        
        network_features = self.feature_network(embedding)
        return F.linear(network_features, self.embedding_weights)


class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()



    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar 
        # (see Project 1a for example)

        # YOUR CODE HERE
        #train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
        optimizer = torch.optim.Adam(self.network.parameters(), lr=1e-3, weight_decay=1e-6)
        for epoch in tqdm.notebook.tqdm(range(10), leave=False):
            for i, (prefix, word_before) in enumerate(tqdm.notebook.tqdm(train_loader, leave=False)):
                #move things to GPU
                prefix = prefix.cuda()
                word_before = word_before.cuda()
                optimizer.zero_grad()
                
                network_logits = self.network(prefix)
                loss = torch.nn.functional.cross_entropy(
                    network_logits, word_before
                )
                loss.backward()
                optimizer.step()



    def next_word_probabilities(self, text_prefix):
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        # the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network
        if len(text_prefix) < self.n - 1:
            text_prefix = ['<eos>'] * (self.n-len(text_prefix)-1) + text_prefix
            
        text_prefix = text_prefix[-(self.n - 1):]
        #you will need to convert text_prefix from strings to numbers with the `ids` function
        prefix = torch.tensor(ids(text_prefix), dtype=torch.int64).cuda().view(1, -1)
        with torch.no_grad():
            #don't forget self.network.eval()
            self.network.eval()
            network_logits = self.network(prefix)
            #call train on network model
            self.network.train()
            probabilities = F.softmax(network_logits, dim=-1).view(-1)
            #move back to cpu when returning
        return probabilities.cpu().numpy()



    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()

        # YOUR CODE HERE
        #Don't forget self.network.eval()
        self.network.eval()
        total_probability = 0
        total_count = 0
        data = NeuralNgramDataset(ids(text), self.n)
        data_loader = torch.utils.data.DataLoader(data, batch_size=128, shuffle=True)
        with torch.no_grad():
            for prefix, word_before in tqdm.notebook.tqdm(data_loader, leave=False):
                #move to gpu
                prefix = prefix.cuda()
                word_before = word_before.cuda()
                network_logits = self.network(prefix)
                probabilities = F.softmax(network_logits, dim=-1)
                selected_probabilities = probabilities[torch.arange(word_before.shape[-1]), word_before]
                selected_probabilities = selected_probabilities.cpu().numpy()
                total_count += word_before.shape[0]
                total_probability += np.sum(np.log(selected_probabilities) / np.log(2))
        #call train on network model
        self.network.train()
        return 2 ** -(total_probability / total_count)


neural_trigram_model = NeuralNGramModel(3)
check_validity(neural_trigram_model)
neural_trigram_model.train()

print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))
save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/32635 [00:00<?, ?it/s]

  0%|          | 0/32635 [00:00<?, ?it/s]

  0%|          | 0/32635 [00:00<?, ?it/s]

...

  0%|          | 0/32635 [00:00<?, ?it/s]

  0%|          | 0/1701 [00:00<?, ?it/s]

neural trigram validation perplexity: 207.0778360130808


  0%|          | 0/5000 [00:00<?, ?it/s]

saved neural_trigram_predictions.npy


Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: ***207.0778360130808***

Free up RAM.

In [17]:
# Delete model we don't need. 
del neural_trigram_model

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We'll use the `torchtext` library to manage this batching for you.  To properly deal with this input format, you should save the last state of the LSTM from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity below 130.  The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size). **NOTE: You may find that adding nonlinearities between these layers can hurt performance, try without first.**
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [18]:
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        # YOUR CODE HERE
        self.embedding_weights = nn.Parameter(
            torch.randn(len(vocab), 128) * np.sqrt(2.0 / 128)
        )  
        # 3 LSTM layers with 512 units
        self.lstm = nn.LSTM(128, 512, 3, dropout=0.5)
        self.dropout = nn.Dropout(p=0.5)
        self.layer_norm1 = nn.LayerNorm(512)
        self.shrink_linear = nn.Linear(512, 1024)
        self.projection = nn.Linear(1024, 128)
        self.layer_norm2 = nn.LayerNorm(128)
        

        self.linear_bias = nn.Parameter(torch.zeros(len(vocab)))
        


    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        embedding = F.embedding(x, self.embedding_weights)
        output_of_lstm, new_state = self.lstm(embedding, state)
        new_state = (new_state[0].detach(), new_state[1].detach())
        x = self.dropout(output_of_lstm)
        x = self.layer_norm1(x)
        x = self.shrink_linear(x)
        x = self.projection(x)
        x = self.layer_norm2(x)
        return F.linear(x, self.embedding_weights, self.linear_bias), new_state


class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.network = LSTMNetwork().cuda()

    def train(self):
        train_iterator = data.BPTTIterator(train_dataset, batch_size=64, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.
      

        # YOUR CODE HERE
        if not hasattr(self, 'optimizer'):
            self.optimizer = torch.optim.Adam(self.network.parameters(), lr=1e-3, weight_decay= 1e-6)
            
        # train with Adam (using default learning rates) for at least 20 epochs
        for epoch in tqdm.notebook.tqdm(range(20), leave=False):

            #The initial state passed into the LSTM should be set to zero.
            state = None
            for i, batch in enumerate(tqdm.notebook.tqdm(train_iterator, leave=False)):
                #move to GPU
                prefix = batch.text.cuda()
                word_before = batch.target.cuda()
                self.optimizer.zero_grad()
                network_logits, state = self.network(prefix, state)
                network_logits = network_logits.view(-1, network_logits.shape[-1])
                word_before = word_before.view(-1)
                loss = torch.nn.functional.cross_entropy(network_logits, word_before)
                loss.backward()
                self.optimizer.step()

            #Print perplexity at every point. Helpfull during debgging 
            validation_perplexity = self.dataset_perplexity(validation_dataset)
            print("Validation perplexity", validation_perplexity)


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
    
        # YOUR CODE HERE

        with torch.no_grad():
            #call eval
            self.network.eval()

            network_logits, _ = self.network(prefix_token_tensor, None)

            probabilities = F.softmax(network_logits, dim=-1)
            # probabilities = probabilities.view(-1, probabilities.shape[-1])
            # print(probabilities.size())
            # move back to cpu when returning
            next_word_prob = probabilities[-1]
            # print(next_word_prob.size())
            next_word_prob = next_word_prob.view(-1)
            # print(next_word_prob.size())
            next_word_prob = next_word_prob.data.cpu().numpy()

        #call train
        self.network.train()
        
        return next_word_prob



    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')

        # YOUR CODE HERE
        #call eval
        self.network.eval()

        #initialize the probability and count
        total_probabilities = 0
        total_count = 0
        
        with torch.no_grad():
            state = None
            for i, batch in enumerate(tqdm.notebook.tqdm(iterator, leave=False)):
                #move things to GPU
                prefix = batch.text.cuda()
                word_before = batch.target.cuda()
                network_logits, state = self.network(prefix, state)
                probabilities = F.softmax(network_logits, dim=-1)
                probabilities = probabilities.view(-1, probabilities.shape[-1])
                word_before = word_before.view(-1)
                selected_probabilities = probabilities[torch.arange(word_before.shape[0]).cuda(), word_before]
                selected_probabilities = selected_probabilities.cpu().numpy()
                # print(selected_probabilities)
                total_count += word_before.shape[0]
                total_probabilities += np.sum(np.log(selected_probabilities) / np.log(2))
        #call train
        self.network.train()
        perplexity = 2 ** (-(total_probabilities / total_count))
        
        return  perplexity


lstm_model = LSTMModel()
lstm_model.train()


print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

Validation perplexity 560.2908839209415


  0%|          | 0/1020 [00:00<?, ?it/s]

...

Validation perplexity 132.98114307399564


  0%|          | 0/107 [00:00<?, ?it/s]

lstm validation perplexity: 132.98114307399564


  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


In [19]:
print(generate_text(lstm_model))

<eos> <eos> = = Discovery and reception = = <eos> <eos> From 1995 onwards , Carre 's shifted the Los Angeles administration


<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Fill in your LSTM perplexity. 

LSTM validation perplexity: ***132.98114307399564***

LSTM improved validation perplexity : ***109.82095241843462***

# Experimentation: 1-Page Report

Now it's time for you to experiment.  Try to reach a validation perplexity below 120. You may either modify the LSTM class above, or copy it down to the code cell below and modify it there. Just **be sure to run code cell below to generate results with your improved LSTM**.  

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing number or sizes of layers), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf):
* activation regularization - add a l2 regularization penalty on the activation of the LSTM output (standard l2 regularization is on the weights)
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* learning rate scheduling - decrease the learning rate during training
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* ensembling - average the predictions of several models trained with different initialization random seeds
* temporal activation regularization - add l2 regularization on the difference between the LSTM output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

For this section, you will submit a write-up describing the extensions and/or modifications that you tried.  Your write-up should be **1-page maximum** in length and should be submitted in PDF format.  You may use any editor you like, but we recommend using LaTeX and working in an environment like Overleaf.
For full credit, your write-up should include:
1.   A concise and precise description of the extension that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing validation perplexities of your improvement to the original LSTM.
The purpose of this exercise is to experiment, so feel free to try/ablate multiple of the suggestions above as well as any others you come up with!
When you submit the file, please name it `report.pdf`.



Run the cell below in order to train your improved LSTM and evaluate it.  

In [20]:
import os
## Feel free to copy your original LSTM solution down here to modify for your report if you'd like. 
# YOUR CODE [optionally] HERE
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        # YOUR CODE HERE
        # 3 LSTM layers with 512 units
        # increased the dimensions of the embedding from 128 and 512
        self.lstm = nn.LSTM(512,512,num_layers=3, dropout = 0.5)
        self.dropout = nn.Dropout(p=0.5)
        self.shrink_linear = nn.Linear(512, 512)
        self.output_linear = nn.Linear(512, vocab_size)


    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        x = F.embedding(x, weight=self.output_linear.weight)
        x, state = self.lstm(x, state)
        x = self.dropout(x)
        x = self.shrink_linear(x)
        x = self.output_linear(x)
        output = F.log_softmax(x, dim=-1)
        return output, state


class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.network = LSTMNetwork().cuda()

    def train(self):
        #increased the batch_size to improve performance
        train_iterator = data.BPTTIterator(train_dataset, batch_size=64, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE
        
        LOSS = torch.nn.NLLLoss()
        if not hasattr(self, 'optimizer'):
            #activation regularization
            # train with Adam (using default learning rates) 
            self.optimizer = torch.optim.Adam(self.network.parameters(), lr=1e-3)
        # Learning rate scheduling 
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[10, 12, 14, 16, 18, 20], gamma=0.2)
        
        #create a directory to keep track of the models
        if not os.path.exists("models"):
            os.makedirs("models")
        
        #hold the best perplexity so far
        best_so_far = float('inf')
        
        # train with Adam (using default learning rates) for at least 20 epochs
        #20 is a lot and takes a while so I decreased it and it actually lowered my perplexity. 
        # I used 10. 
        for epoch in tqdm.notebook.tqdm(range(20), leave=False):
            print("Epoch", epoch)
            #set the state
            h_0 = torch.zeros(3, 64, 512).cuda()
            c_0 = torch.zeros(3, 64, 512).cuda()
            state_0 = (h_0, c_0)
            
            for i, batch in enumerate(tqdm.notebook.tqdm(train_iterator, leave=False)):
                #move to GPU
                prefix = batch.text.cuda()
                word_before = batch.target.cuda()
                self.optimizer.zero_grad()
                network_logits, (h_t, c_t) = self.network(prefix, state_0)
                loss = LOSS(network_logits.view(-1,vocab_size), word_before.flatten())
                loss.backward()
                self.optimizer.step()
                state_0 = (h_t.detach(), c_t.detach())
            
            #keep track of the perplexity and only change it if its less than the last one 
            validation_perplexity = self.dataset_perplexity(validation_dataset)
            print("Validation perplexity", validation_perplexity)
                                        
            if validation_perplexity < best_so_far:
                best_so_far = validation_perplexity
                torch.save(self.network.state_dict(), "models/lstm_model")

            # Learning rate scheduling step                       
            lr_scheduler.step()

            self.network.load_state_dict(torch.load("models/lstm_model"))



    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        
        # YOUR CODE HERE
        #call eval
        self.network.eval()
        
        with torch.no_grad():
            
            network_logits, _ = self.network(prefix_token_tensor, None)
        
      
            probabilities = F.softmax(network_logits, dim=-1)
            next_word_prob = probabilities[-1]
            # print(next_word_prob.size())
            next_word_prob = next_word_prob.view(-1)
            next_word_prob = next_word_prob.data.cpu().numpy()
                                        
        #call train
        self.network.train()
        return next_word_prob



    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')

        # YOUR CODE HERE
        #call eval
        self.network.eval()
        #set the state
        h_0 = torch.zeros(3, 64, 512).cuda()
        c_0 = torch.zeros(3, 64, 512).cuda()
        state_0 = (h_0, c_0)
        
        total_probabilities = []
        
        for i, batch in enumerate(tqdm.notebook.tqdm(iterator, leave=False)):            
            prefix = batch.text.cuda()
            word_before = batch.target.cuda()
            
            with torch.no_grad():
                network_logits, (h_t, c_t) = self.network(prefix, state_0)
                state_0 = (h_t.detach(), c_t.detach())
            
            selected = network_logits.gather(-1, word_before.unsqueeze(-1))
            selected_list = selected.flatten().tolist()
            total_probabilities.extend(selected_list)
            
        #call train
        self.network.train()
        return np.exp(-np.mean(total_probabilities))


lstm_model = LSTMModel()
lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0


  0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 0/107 [00:00<?, ?it/s]

Validation perplexity 264.19359349402913
Epoch 1


...

Validation perplexity 109.82095241843462


  0%|          | 0/107 [00:00<?, ?it/s]

lstm validation perplexity: 109.82095241843462


  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


In [21]:
##improved LSTM = 109.82095241843462

In [25]:
print(generate_text(lstm_model))

<eos> <eos> = = Name = = <eos> <eos> There was many exceptions , the death of the Earth 's main registry


### Submission

Upload a submission with the following files to Gradescope:
* hw1b.ipynb (rename to match this exactly)
* lstm_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* bigram_predictions.npy
* report.pdf

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.